## Imports

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from tqdm import tqdm
from tsai.all import *

from torch.utils.data.dataloader import DataLoader

from trace_process import *

## Constants and Sizes

In [2]:
TRAIN_SIZE = 20_000
TEST_SIZE = 5_000
TIME_DELTA = 2000 * NANO_TO_MICRO
BATCH_SIZE = 32
MTU = 1514
WINDOW_SIZE = 10

TRAIN_PATH = "../data/15/node-2/train/packets"
TEST_PATH = "../data/15/node-2/test/packets"

## Forging Data

### Definitions

In [3]:
def get_packets(file_path):
    df = pd.read_csv(
        file_path, 
        header=None,
        index_col=False,
        names=['timestamp', 'size', 'src', 'dest', 'dir'], 
        dtype={'size': "int16", 'src': "category", 'dest': "category", "timestamp": "int64", "size": "int64", "dir": "int8"},
    )
    df = df[df['dir'] == 1][["timestamp", "size"]]
    df = df.sort_values(by='timestamp')
    return df.values

In [4]:
class TraceDataset(torch.utils.data.Dataset):
    def __init__(self, path, test=False):
        raw_vals = get_packets(path)
        self.is_testing = test
        vals, times, sizes, _ = get_flow_trace_time(raw_vals, TIME_DELTA)
        sizes = np.roll(sizes, -1)
        self.vals = vals
        self.times = times
        self.sizes = sizes
        self.threshold = np.median(sizes)
        self.max_len = np.max([len(f) for f in self.times])
        self.labels = torch.Tensor((sizes > self.threshold) * 1)
    def __len__(self):
        if self.is_testing:
            return 1000
        else:
            return 10000
            #return len(self.times)
    def __getitem__(self, idx):
        t1 = torch.Tensor(self.vals[idx])
        t2 = torch.Tensor(self.times[idx])
        t1 = F.pad(t1, (self.max_len - t1.shape[0], 0))
        t2 = F.pad(t2, (self.max_len - t2.shape[0], 0))
        return torch.cat((t1.unsqueeze(1), t2.unsqueeze(1)), dim=1), self.labels[idx]

In [5]:
train_data = DataLoader(TraceDataset(TRAIN_PATH), batch_size=BATCH_SIZE, drop_last=True, shuffle=True)
test_data = DataLoader(TraceDataset(TEST_PATH, test=True), batch_size=BATCH_SIZE, drop_last=True, shuffle=True)

In [6]:
class TraceModel(torch.nn.Module):
    def __init__(self, input_size, hidden_units, output_size, num_layers):
        super().__init__()
        self.input_size = input_size
        self.hidden_units = hidden_units
        self.num_layers = num_layers
        self.output_size = output_size

        self.lstm = torch.nn.LSTM(input_size=self.input_size, hidden_size=self.hidden_units, batch_first=True, num_layers=self.num_layers)
        self.linear = torch.nn.Linear(in_features=self.hidden_units, out_features=output_size)


    def forward(self, x):
        h0 = torch.zeros(self.num_layers, BATCH_SIZE, self.hidden_units).cuda()
        c0 = torch.zeros(self.num_layers, BATCH_SIZE, self.hidden_units).cuda()
        output, (hn, cn) = self.lstm(x, (h0, c0)) # preserve states?
        out = self.linear(output[:, -1]).flatten()

        return F.sigmoid(out)

In [7]:
input_dim = 2    
hidden_dim = 256
layer_dim = 3
output_dim = 1 

lr = 0.0005
n_epochs = 10
iterations_per_epoch = len(train_data)
best_acc = 0

model = MLSTM_FCN(c_in=2, c_out=1, seq_len=5906, shuffle=False) # TraceModel(input_dim, hidden_dim, output_dim, layer_dim)
model = model.cuda()
criterion = torch.nn.BCELoss()
opt = torch.optim.Adam(model.parameters(), lr=lr)
# sched = torch.optim.CyclicLR(opt, cosine(t_max=iterations_per_epoch * 2, eta_min=lr/100))

print('Start model training')

for epoch in range(1, n_epochs + 1):
    
    for i, (x_batch, y_batch) in enumerate(tqdm(train_data)):
        model.train()
        x_batch = x_batch.permute((0, 2, 1))
        x_batch = x_batch.cuda()
        y_batch = y_batch.cuda()
        # sched.step()
        opt.zero_grad()
        out = model(x_batch)
        out = F.sigmoid(out.view(-1))
        loss = criterion(out, y_batch)
        loss.backward()
        opt.step()
    
    model.eval()
    correct, total = 0, 0
    for x_val, y_val in test_data:
        x_val = x_val.permute((0, 2, 1))
        x_val, y_val = [t.cuda() for t in (x_val, y_val)]
        out = model(x_val)
        out = F.sigmoid(out.view(-1))
        preds = torch.round(out)
        total += y_val.size(0)
        correct += (preds == y_val).sum().item()
    
    acc = correct / total

    print(f'Epoch: {epoch:3d}. Loss: {loss.item():.4f}. Acc.: {acc:2.2%}')

    if acc > best_acc:
        trials = 0
        best_acc = acc
        torch.save(model.state_dict(), 'best.pth')
        print(f'Epoch {epoch} best model saved with accuracy: {best_acc:2.2%}')


Start model training


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 312/312 [01:59<00:00,  2.61it/s]


RuntimeError: CUDA out of memory. Tried to allocate 948.00 MiB (GPU 0; 11.75 GiB total capacity; 8.14 GiB already allocated; 170.69 MiB free; 9.28 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF